In [19]:
import numpy as np
import random


def cal_time_employee(segment, d, t):
    if len(segment) == 0:
        return 0
    time_work = 0
    time_work += t[0][segment[0]]
    for i in range(len(segment) - 1):
        time_work += t[segment[i]][segment[i+1]] + d[segment[i]]
    time_work += t[segment[-1]][0] + d[segment[-1]]
    return time_work

def split_segment(chromosome, N):
    segment = []
    segment_employee = []
    for i in range(len(chromosome)):
        if chromosome[i] > N:
            segment.append(segment_employee)
            segment_employee = []
        else:
            segment_employee.append(chromosome[i])
    segment.append(segment_employee)
    return segment
        


class Individual:
    def __init__(self, N, K, d, t):
        self.chromosome = list(np.zeros(N + K - 1, dtype = int))
        self.fitness = None
        self.N = N
        self.K = K
        self.d = d
        self.t = t
    
    def cal_fitness(self):
        segment = split_segment(self.chromosome, self.N)
        max_time = 0
        for i in range (self.K):
            max_time = max(max_time, cal_time_employee(segment[i], self.d, self.t))
        self.fitness = max_time
        
    def gen_random(self):
        number_list = list(range(1, self.N+self.K))
        self.chromosome = list(random.sample(number_list, len(number_list)))
        self.cal_fitness()

    
    
    
class Population:
    def __init__(self, N, K, d, t, POP_SIZE = 100, MAX_GEN = 500, crossover_parameter = 0.8, mutation_parameter = 0.1):
        self.indivs = []
        self.N = N
        self.K = K
        self.d = d
        self.t = t
        self.POP_SIZE = POP_SIZE
        self.MAX_GEN = MAX_GEN
        self.crossover_parameter = crossover_parameter
        self.mutation_parameter = mutation_parameter
    
    def gen_pop(self):
        for i in range(self.POP_SIZE):
            indi = Individual(self.N, self.K, self.d, self.t)
            indi.gen_random()
            self.indivs.append(indi)

    def order_crossover(self, parent1, parent2):
        indi1 = Individual(self.N, self.K, self.d, self.t)
        indi2 = Individual(self.N, self.K, self.d, self.t)
        
        start = random.randint(0, self.N + self.K - 3)
        end = random.randint(start, self.N + self.K -2)
        for i in range(start, end + 1):
            indi1.chromosome[i] = parent1.chromosome[i]
            indi2.chromosome[i] = parent2.chromosome[i]
        
        indi1_index = end + 1
        indi1_i = 0
        while indi1_index < self.N + self.K - 1:
            if parent2.chromosome[indi1_i] not in indi1.chromosome:
                indi1.chromosome[indi1_index] = parent2.chromosome[indi1_i]
                indi1_index += 1
            indi1_i += 1
        indi1_index = 0
        while indi1_index < start:
            if parent2.chromosome[indi1_i] not in indi1.chromosome:
                indi1.chromosome[indi1_index] = parent2.chromosome[indi1_i]
                indi1_index += 1
            indi1_i += 1
        
        indi2_index = end + 1
        indi2_i = 0
        while indi2_index < self.N + self.K - 1:
            if parent1.chromosome[indi2_i] not in indi2.chromosome:
                indi2.chromosome[indi2_index] = parent1.chromosome[indi2_i]
                indi2_index += 1
            indi2_i += 1
        
        indi2_index = 0
        while indi2_index < start:
            if parent1.chromosome[indi2_i] not in indi2.chromosome:
                indi2.chromosome[indi2_index] = parent1.chromosome[indi2_i]
                indi2_index += 1
            indi2_i += 1
        
        indi1.cal_fitness()
        indi2.cal_fitness()
        return indi1, indi2

    def mutation(self, parent):
        indi = Individual(self.N, self.K, self.d, self.t)
        index1 = random.randint(0, self.N + self.K -2)
        index2 = random.randint(0, self.N + self.K -2)
        while index1 == index2:
            index2 = random.randint(0, self.N + self.K -2)
        
        indi.chromosome = parent.chromosome.copy()
        indi.chromosome[index1], indi.chromosome[index2] = indi.chromosome[index2], indi.chromosome[index1]
        indi.cal_fitness()
        return indi
        
    def evolve(self):
        for i in range(self.MAX_GEN):
            for j in range(self.POP_SIZE):
                if random.random() < self.crossover_parameter:
                    parent1_index = random.randint(0, self.POP_SIZE - 1)
                    parent2_index = random.randint(0, self.POP_SIZE - 1)
                    while parent1_index == parent2_index:
                        parent2_index = random.randint(0, self.POP_SIZE - 1)
                    parent1 = self.indivs[parent1_index]
                    parent2 = self.indivs[parent2_index]
                    
                    indi1, indi2 = self.order_crossover(parent1, parent2)
                    if random.random() < self.mutation_parameter:
                        indi1 = self.mutation(indi1)
                        indi2 = self.mutation(indi2)
                    self.indivs.append(indi1)
                    self.indivs.append(indi2)
            self.indivs.sort(key = lambda x: x.fitness)
            self.indivs = self.indivs[:self.POP_SIZE]
        return self.indivs[0].fitness, self.indivs[0].chromosome


In [20]:
def read_data(file_data):
    data_path = "data\\" + file_data
    with open(data_path, 'r') as f:
        N, K = [int(x) for x in f.readline().split()]
        d = [int(x) for x in f.readline().split()]
        d.insert(0, 0)
        t = []
        for i in range(N + K):
            t.append([int(x) for x in f.readline().split()])
    return N, K, d, t

In [21]:
import time
def ga(file_data):
    N, K, d, t = read_data(file_data)
    time_start = time.time()
    pop = Population(N, K, d, t)
    pop.gen_pop()
    fitness, chromosome = pop.evolve()
    segment = split_segment(chromosome, N)
    time_pro = time.time() - time_start
    result_path = "GA\\1_" + file_data
    with open(result_path, 'w') as f:
        f.write("Objective: " + str(fitness) + '\n')
        f.write("Time: " + str(time_pro) + '\n')
        f.write(str(K) + '\n')
        for se in segment:
            f.write(str(len(se) + 2) + '\n')
            f.write('0 ')
            for pos in se:
                f.write(str(pos) + ' ')
            f.write('0\n')
    print(K)
    for se in segment:
        print(len(se) + 2)
        print(0, end = ' ')
        for pos in se:
            print(pos, end = ' ')
        print(0)
    print(fitness)

In [22]:
data = ["data_5_2.txt", "data_5_3.txt", "data_10_2.txt", "data_10_3.txt","data_10_5.txt", "data_15_3.txt", "data_15_5.txt", "data_15_7.txt", "data_20_5.txt", "data_20_7.txt","data_20_10.txt", "data_50_5.txt", "data_50_10.txt", "data_50_20.txt", "data_100_10.txt", "data_200_10.txt", "data_200_20.txt", "data_400_40.txt", "data_500_50.txt","data_700_70.txt", "data_900_90.txt", "data_1000_100.txt"]

In [23]:
for file_data in data:
    ga(file_data)

2
4
0 3 5 0
5
0 4 1 2 0
360
3
4
0 1 2 0
3
0 5 0
4
0 3 4 0
861
2
6
0 3 4 7 1 0
8
0 2 6 5 8 9 10 0
540
3
6
0 2 8 10 1 0
5
0 5 7 6 0
5
0 3 4 9 0
1898
5
3
0 1 0
4
0 4 8 0
5
0 5 9 3 0
4
0 6 2 0
4
0 10 7 0
1193
3
7
0 13 7 3 12 14 0
8
0 5 11 10 1 9 4 0
6
0 6 8 2 15 0
3645
5
6
0 5 12 15 13 0
5
0 4 7 14 0
5
0 8 1 3 0
4
0 10 6 0
5
0 11 2 9 0
1831
7
4
0 5 13 0
4
0 12 11 0
4
0 14 7 0
4
0 15 9 0
5
0 4 8 3 0
4
0 6 1 0
4
0 2 10 0
1527
5
6
0 9 8 12 11 0
5
0 6 2 19 0
7
0 15 18 4 14 16 0
6
0 5 20 7 13 0
6
0 3 1 10 17 0
2359
7
4
0 10 19 0
5
0 3 9 1 0
4
0 11 16 0
5
0 13 12 6 0
4
0 7 2 0
6
0 20 17 8 14 0
6
0 15 5 4 18 0
1475
10
4
0 7 13 0
4
0 18 14 0
4
0 9 19 0
4
0 20 15 0
4
0 16 10 0
4
0 6 12 0
4
0 11 3 0
5
0 17 8 4 0
3
0 1 0
4
0 2 5 0
1836
5
12
0 43 31 25 33 8 3 30 42 23 24 0
14
0 44 49 36 35 2 47 45 39 17 7 6 37 0
12
0 28 12 9 19 5 13 29 18 46 4 0
11
0 1 32 50 11 20 15 48 26 38 0
11
0 21 10 16 41 22 14 34 27 40 0
6926
10
6
0 25 16 24 45 0
8
0 47 5 49 27 11 18 0
9
0 21 39 19 7 40 41 3 0
7
0 13 23 31 33 2